In [20]:
import torch as th
import numpy as np
from ActiveCritic.tests.test_utils.utils import make_wsm_setup
from ActiveCritic.policy.active_critic_policy import ActiveCriticPolicy, ActiveCriticPolicySetup, ACPOptResult
from ActiveCritic.model_src.whole_sequence_model import WholeSequenceActor, WholeSequenceCritic, WholeSequenceModelSetup
from ActiveCritic.model_src.transformer import TransformerModel, CriticTransformer, ModelSetup
from ActiveCritic.utils.gym_utils import new_epoch, DummyExtractor
import gym


In [21]:
def make_obs_act_space(obs_dim, action_dim):
    obs_array_low = [0]*obs_dim
    obs_array_high = [1]*obs_dim
    action_low = [0]*action_dim
    action_high = [1]*action_dim
    observation_space = gym.spaces.box.Box(
        np.array(obs_array_low), np.array(obs_array_high), (obs_dim,), float)
    # next state (4)
    action_space = gym.spaces.box.Box(
        np.array(action_low), np.array(action_high), (action_dim,), float)
    return observation_space, action_space

In [22]:
seq_len = 4
d_output = 2
d_result = 1

wsm_actor_setup = make_wsm_setup(seq_len=seq_len, d_output=d_output, model_class=TransformerModel)
wsm_critic_setup = make_wsm_setup(seq_len=seq_len, d_output=d_output, model_class=CriticTransformer, d_result=d_result)

In [23]:
actor = WholeSequenceActor(wsm_actor_setup)
critic = WholeSequenceCritic(wsm_critic_setup)

In [24]:
obs, acts = make_obs_act_space(4, 2)

/home/hendrik/anaconda3/envs/tfTest/lib/python3.9/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float64
  logger.warn(


In [29]:
def make_acps(seq_len, extractor, new_epoch):
    acps = ActiveCriticPolicySetup()
    acps.device='cuda'
    acps.epoch_len=seq_len
    acps.extractor=extractor
    acps.new_epoch=new_epoch
    acps.opt_steps=2
    acps.optimisation_threshold=0.9
    return acps

acps = make_acps(seq_len=seq_len, extractor=DummyExtractor(), new_epoch=new_epoch)

In [30]:
ac = ActiveCriticPolicy(observation_space=obs, action_space=acts, actor=actor, critic=critic, return_mode=0, acps=acps)

In [31]:
obs = th.ones([2, 4, 3], dtype=th.float, device='cuda')
next_action = ac.predict(obs)

RuntimeError: expand(torch.cuda.FloatTensor{[2, 1, 4, 3]}, size=[2, 1, 4]): the number of sizes provided (3) must be greater or equal to the number of dimensions in the tensor (4)